In [1]:
# embedding_type = "V3Large"
embedding_type = "ada"

In [2]:
import pandas as pd
import json

with open(file=f"../result/embeddings/{embedding_type}.json", mode="r") as f:
    data = json.load(f)

df = pd.json_normalize(data, max_level=1).T  
  
# Print the DataFrame  
df.head(2)

,0
1.Get close to bottle,"{'closest_pattern': 'Stay close to bottle', 'p..."
1.turn right,"{'closest_pattern': 'Go to the right', 'predic..."


In [3]:
with open(file=f"../result/embeddings/{embedding_type}.json", mode="r") as f:
    data = json.load(f)

corrections = {}
for id in data:
    correction = data[str(id)].pop("correction") 
    corrections[id] = correction


In [4]:
df_corr = pd.DataFrame.from_dict(corrections, orient='columns').T
df_corr.reset_index(inplace=True)  
df_corr.head(2)

,index,predicted,truth
0,1,NaN,NaN
1,2,NaN,NaN


In [5]:
rows = []  
for id, values in data.items():  
    segment = 1  
    for cmd, cmd_values in values.items():
        if isinstance(cmd_values, dict):  
            row = {"id": id,"segment": segment, "cmd": cmd}  
            row.update(cmd_values)  
            rows.append(row)  
            segment += 1 
  
df = pd.DataFrame(rows) 
df.head(3)

,id,segment,cmd,closest_pattern,predicted_feature,confidence_score
0,1,1,Get close to bottle,Stay close to bottle,bottle distance decrease,0.95649
1,1,2,turn right,Go to the right,X-cartesian increase,0.91407
2,1,3,move closer to apple,Move closer to apple,apple distance decrease,0.98758


In [6]:
new_df = pd.merge(df, df_corr, left_on='id', right_on='index', how='left')
new_df.drop(columns=['index'], inplace=True)
print(len(new_df))
new_df.head(3)

192


,id,segment,cmd,closest_pattern,predicted_feature,confidence_score,predicted,truth
0,1,1,Get close to bottle,Stay close to bottle,bottle distance decrease,0.95649,NaN,NaN
1,1,2,turn right,Go to the right,X-cartesian increase,0.91407,NaN,NaN
2,1,3,move closer to apple,Move closer to apple,apple distance decrease,0.98758,NaN,NaN


In [7]:
new_df['id'] = new_df['id'].astype(int)  
new_df['predicted'] = new_df['predicted'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)  
new_df['truth'] = new_df['truth'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)  
  
df = new_df.groupby('id', as_index=False).agg({  
    'segment': list,  
    'cmd': list,  
    'closest_pattern': list,  
    'predicted_feature': list,  
    'confidence_score': list,  
    'predicted': 'first',  
    'truth': 'first'  
})  

df.sort_values(by='id', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

df.head(6)

,id,segment,cmd,closest_pattern,predicted_feature,confidence_score,predicted,truth
0,1,"[1, 2, 3, 4]","[Get close to bottle, turn right, move closer ...","[Stay close to bottle, Go to the right, Move c...","[bottle distance decrease, X-cartesian increas...","[0.95649, 0.91407, 0.98758, 0.98614]",None,None
1,2,"[1, 2, 3, 4]","[Approach the vase, move left, get nearer to t...","[Move closer to vase, Move left, Move closer t...","[vase distance decrease, X-cartesian decrease,...","[0.917, 0.97668, 0.94729, 0.97344]",None,None
2,3,"[1, 2, 3, 4]","[Reduce the distance to the box, turn left, ge...","[Decrease distance to box, Go to the left, Mov...","[box distance decrease, X-cartesian decrease, ...","[0.96696, 0.92288, 0.93717, 0.94436]",None,None
3,4,"[1, 2, 3, 4]","[Move closer to the cup, go right, keep close ...","[Move closer to cup, Go to the right, Stay clo...","[cup distance decrease, X-cartesian increase, ...","[0.99293, 0.91925, 0.97278, 0.91764]",None,None
4,5,"[1, 2, 3, 4]","[Stay close to the jar, move leftward, decreas...","[Stay close to jar, Move left, Decrease distan...","[jar distance decrease, X-cartesian decrease, ...","[0.98351, 0.91491, 0.98107, 0.9618]",None,None
5,6,"[1, 2, 3, 4]","[Get nearer to the bottle, approach the monito...","[Move closer to bottle, Stay close to monitor,...","[bottle distance decrease, monitor distance de...","[0.95869, 0.90662, 0.87411, 0.97344]",None,None


In [8]:
df.to_excel(f"../result/embeddings/{embedding_type}.xlsx", index=False)

In [9]:
import pandas as pd

df = pd.read_excel(f"../result/embeddings/{embedding_type}.xlsx")
df.set_index('id', inplace=True)
df['match'] = df['truth'].isna()
df.head(6)

,segment,cmd,closest_pattern,predicted_feature,confidence_score,predicted,truth,match
id,,,,,,,,
1,"[1, 2, 3, 4]","['Get close to bottle', 'turn right', 'move cl...","['Stay close to bottle', 'Go to the right', 'M...","['bottle distance decrease', 'X-cartesian incr...","[0.95649, 0.91407, 0.98758, 0.98614]",NaN,NaN,True
2,"[1, 2, 3, 4]","['Approach the vase', 'move left', 'get nearer...","['Move closer to vase', 'Move left', 'Move clo...","['vase distance decrease', 'X-cartesian decrea...","[0.917, 0.97668, 0.94729, 0.97344]",NaN,NaN,True
3,"[1, 2, 3, 4]","['Reduce the distance to the box', 'turn left'...","['Decrease distance to box', 'Go to the left',...","['box distance decrease', 'X-cartesian decreas...","[0.96696, 0.92288, 0.93717, 0.94436]",NaN,NaN,True
4,"[1, 2, 3, 4]","['Move closer to the cup', 'go right', 'keep c...","['Move closer to cup', 'Go to the right', 'Sta...","['cup distance decrease', 'X-cartesian increas...","[0.99293, 0.91925, 0.97278, 0.91764]",NaN,NaN,True
5,"[1, 2, 3, 4]","['Stay close to the jar', 'move leftward', 'de...","['Stay close to jar', 'Move left', 'Decrease d...","['jar distance decrease', 'X-cartesian decreas...","[0.98351, 0.91491, 0.98107, 0.9618]",NaN,NaN,True
6,"[1, 2, 3, 4]","['Get nearer to the bottle', 'approach the mon...","['Move closer to bottle', 'Stay close to monit...","['bottle distance decrease', 'monitor distance...","[0.95869, 0.90662, 0.87411, 0.97344]",NaN,NaN,True


In [10]:
single_idx = [29, 30, 31, 32, 33, 34, 35, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85, 87, 88, 89, 90, 96, 100]

print(len(single_idx))

single_cmd = df.loc[df.index.isin(single_idx)]
chain_cmd = df.loc[~df.index.isin(single_idx)]

print(f"Number of single command: {len(single_cmd)}")
print(f"Number of chain command: {len(chain_cmd)}")

55
Number of single command: 55
Number of chain command: 45


In [11]:
value_counts_dict = {
    'single_cmd': single_cmd['match'].value_counts(),
    'chain_cmd': chain_cmd['match'].value_counts()
}
df_stat = pd.DataFrame(value_counts_dict)
df_stat = df_stat.fillna(0)

df_stat

,single_cmd,chain_cmd
match,,
False,0.0,2
True,55.0,43


In [12]:
markdown_table = df.to_markdown(buf=None, mode='pipe', index=True)
res_table = df_stat.to_markdown(buf=None, mode='pipe', index=True)
with open(f'../result/embeddings/{embedding_type}.md', 'w') as file:
    file.write("## Result - ADA (with prompts)\n\n")
    file.write(markdown_table)
    file.write("\n\n")
    file.write("## Stats\n\n")
    file.write(res_table)